# ING Lion's Den 2024


## Team: RiskBusters

 Michał Bryzik, Michał Niegierewicz, Kacper Gruca, Jan Ślusarek

Script to clean testing sample.

We assume that the testing sample has the same structure and issues as the development sample, however we want to check and anlyze this.

We copy-pasted the code to clean development sample and run on testing sample both to clean the dataset and analyze it.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [2]:
data=pd.read_csv('../data/input/testing_sample.csv') 
data.head() 

,ID,customer_id,application_date,target,Application_status,Var1,Var2,Var3,Var4,Var5,...,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,_r_
0,36034977,32653719,03Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4800,15,...,8,8,0,4977.87,14880.28,0,0,70,3899,0.485674
1,36034978,32832365,04Feb2010 0:00:00,0.0,Approved,2,2.0,1.0,6800,18,...,1,1,29400,7922.90,6534.84,0,0,0,3899,0.267045
2,36034979,32544742,07Feb2010 0:00:00,0.0,Approved,1,3.0,1.0,4600,18,...,0,0,0,3174.15,16974.69,0,0,0,3899,0.793579
3,36034980,32395830,09Feb2010 0:00:00,NaN,Rejected,2,2.0,2.0,7900,48,...,2,2,0,4972.82,3484.36,0,0,10,3899,0.533040
4,36034981,32592943,13Feb2010 0:00:00,0.0,Approved,2,1.0,1.0,9900,63,...,3,3,0,2278.22,9630.53,0,0,10,3899,0.781269


In [3]:
data.shape 

(5000, 36)

In [4]:
data.isnull().sum() 

ID                       0
customer_id              0
application_date         0
target                1353
Application_status       0
Var1                     0
Var2                   139
Var3                   139
Var4                     0
Var5                     0
Var6                     0
Var7                     0
Var8                  2905
Var9                     0
Var10                 3719
Var11                    0
Var12                 3719
Var13                    0
Var14                    0
Var15                    0
Var16                    0
Var17                    0
Var18                 3721
Var19                 2905
Var20                    0
Var21                    0
Var22                    0
Var23                    0
Var24                    0
Var25                 1023
Var26                 1961
Var27                    0
Var28                    0
Var29                    0
Var30                    0
_r_                      0
dtype: int64

In [5]:
data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   customer_id         5000 non-null   int64  
 2   application_date    5000 non-null   object 
 3   target              3647 non-null   float64
 4   Application_status  5000 non-null   object 
 5   Var1                5000 non-null   int64  
 6   Var2                4861 non-null   float64
 7   Var3                4861 non-null   float64
 8   Var4                5000 non-null   int64  
 9   Var5                5000 non-null   int64  
 10  Var6                5000 non-null   int64  
 11  Var7                5000 non-null   float64
 12  Var8                2095 non-null   float64
 13  Var9                5000 non-null   int64  
 14  Var10               1281 non-null   float64
 15  Var11               5000 non-null   int64  
 16  Var12 

## Clean dataset

### Define columns by datetypes and sort df

We define the target type of the column basing on the description file and exploring data

In [6]:
# Define columns
date_columns = ['application_date', 'Var13'] 
numeric_columns = ['ID', 'customer_id', '_r_', 'Var9', 'Var10']  
categorical_columns = []
binary_columns = ['target', 'Application_status'] 

In [7]:
# Checking the data type for the rest of columns 
for column in data.columns:
    if column in date_columns + numeric_columns + binary_columns:
        continue  
    elif column in ['Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']:
        numeric_columns.append(column)
    elif column in ['Var2', 'Var3', 'Var11', 'Var12', 'Var14']:
        categorical_columns.append(column)
    elif column in ['Var18', 'Var19', 'Var27', 'Var28']:
        binary_columns.append(column)

In [8]:
print(date_columns)
print(numeric_columns)
print(categorical_columns)
print(binary_columns)

['application_date', 'Var13']
['ID', 'customer_id', '_r_', 'Var9', 'Var10', 'Var1', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'Var15', 'Var16', 'Var17', 'Var20', 'Var21', 'Var22', 'Var23', 'Var24', 'Var25', 'Var26', 'Var29', 'Var30']
['Var2', 'Var3', 'Var11', 'Var12', 'Var14']
['target', 'Application_status', 'Var18', 'Var19', 'Var27', 'Var28']


In [9]:
# Make a copy of main data, from this moment we will work on this dataset
df = data.copy()

In [10]:
# Assumed sequence for columns sorting 
sorted_columns = date_columns + numeric_columns + categorical_columns + binary_columns

# Choose of the sorted value by defined categories
df = data[sorted_columns]

In [11]:
df.head()
# Some of the records for Var14 are Nulls, it requires analysis

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,03Feb2010 0:00:00,28Nov2004,36034977,32653719,0.485674,4200,NaN,1,4800,15,...,1.0,2,NaN,0,0.0,Approved,NaN,NaN,0,0
1,04Feb2010 0:00:00,19Sep1996,36034978,32832365,0.267045,5880,3700.0,2,6800,18,...,1.0,5,6.0,3,0.0,Approved,1.0,NaN,0,0
2,07Feb2010 0:00:00,23Nov1998,36034979,32544742,0.793579,4200,NaN,1,4600,18,...,1.0,5,NaN,0,0.0,Approved,NaN,NaN,0,0
3,09Feb2010 0:00:00,13Nov2005,36034980,32395830,0.533040,4200,5400.0,2,7900,48,...,2.0,5,5.0,2,NaN,Rejected,0.0,NaN,0,0
4,13Feb2010 0:00:00,06Aug1995,36034981,32592943,0.781269,3120,3300.0,2,9900,63,...,1.0,4,5.0,1,0.0,Approved,NaN,0.0,0,0


### Handle date variables

In [12]:
# Print date columns
date_columns

['application_date', 'Var13']

#### Change the type of columns

In [13]:
# Change date columns to datetime type
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['Var13'] = pd.to_datetime(df['Var13'], format='%d%b%Y', errors='coerce')         # Var13 - irregular date format

In [14]:
# Check naN values after transformation
print(df['Var13'].isnull().sum(0))
print(df['application_date'].isnull().sum(0))

49
0


After transformation some records for Var13 are empty, we need to analyze it

In [15]:
df.head()

,application_date,Var13,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,2010-02-03,2004-11-28,36034977,32653719,0.485674,4200,NaN,1,4800,15,...,1.0,2,NaN,0,0.0,Approved,NaN,NaN,0,0
1,2010-02-04,1996-09-19,36034978,32832365,0.267045,5880,3700.0,2,6800,18,...,1.0,5,6.0,3,0.0,Approved,1.0,NaN,0,0
2,2010-02-07,1998-11-23,36034979,32544742,0.793579,4200,NaN,1,4600,18,...,1.0,5,NaN,0,0.0,Approved,NaN,NaN,0,0
3,2010-02-09,2005-11-13,36034980,32395830,0.533040,4200,5400.0,2,7900,48,...,2.0,5,5.0,2,NaN,Rejected,0.0,NaN,0,0
4,2010-02-13,1995-08-06,36034981,32592943,0.781269,3120,3300.0,2,9900,63,...,1.0,4,5.0,1,0.0,Approved,NaN,0.0,0,0


#### Analyze Var13

In [16]:
# Check indexes where conversion caused 'NaT' values
nan_indices = df['Var13'].isna()

# Compare original values of 'Var13' which after conversion became 'NaT'
original_values_with_nan = data.loc[nan_indices, 'Var13']

original_values_with_nan

19      31Dec9999
78      31Dec9999
154     31Dec9999
284     31Dec9999
350     31Dec9999
375     31Dec9999
380     31Dec9999
548     31Dec9999
820     31Dec9999
833     31Dec9999
999     31Dec9999
1143    31Dec9999
1166    31Dec9999
1208    31Dec9999
1346    31Dec9999
1447    31Dec9999
1487    31Dec9999
1514    31Dec9999
1687    31Dec9999
2142    31Dec9999
2338    31Dec9999
2361    31Dec9999
2374    31Dec9999
2415    31Dec9999
2527    31Dec9999
2555    31Dec9999
2754    31Dec9999
2854    31Dec9999
3080    31Dec9999
3212    31Dec9999
3289    31Dec9999
3343    31Dec9999
3355    31Dec9999
3569    31Dec9999
3575    31Dec9999
3608    31Dec9999
3706    31Dec9999
3965    31Dec9999
4056    31Dec9999
4096    31Dec9999
4225    31Dec9999
4267    31Dec9999
4348    31Dec9999
4485    31Dec9999
4584    31Dec9999
4641    31Dec9999
4677    31Dec9999
4753    31Dec9999
4988    31Dec9999
Name: Var13, dtype: object

In [17]:
# Check unique values of 'Var13' which after conversion became 'NaT'
original_values_with_nan.unique()

array(['31Dec9999'], dtype=object)

Summarizing we have 784 cases in Var13 with value '31Dec9999'

We treat them as data error and for this moment leave with NAs

#### Create working_months column

Var13 = employment date

working_moths = Var13 - application_date


We would like to calculate the number of working months before applying for credit, which will indicate the employee's duration at the given position from employment to loan application.

NAs from Var13 are replaced with 0

We also want to have this column first

In [18]:
# Create a new colum working_months
df['working_months'] = ((df['application_date'] - df['Var13']) / np.timedelta64(1, 'M')).fillna(0).astype(int)

In [19]:
# Delete columns 'Var13' and 'application_date'
df.drop(['Var13'], axis=1, inplace=True)
df.drop(['application_date'], axis=1, inplace=True)

In [20]:
df.head()

,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,Var7,...,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28,working_months
0,36034977,32653719,0.485674,4200,NaN,1,4800,15,1,463.31,...,2,NaN,0,0.0,Approved,NaN,NaN,0,0,62
1,36034978,32832365,0.267045,5880,3700.0,2,6800,18,6,3170.40,...,5,6.0,3,0.0,Approved,1.0,NaN,0,0,160
2,36034979,32544742,0.793579,4200,NaN,1,4600,18,1,355.36,...,5,NaN,0,0.0,Approved,NaN,NaN,0,0,134
3,36034980,32395830,0.533040,4200,5400.0,2,7900,48,1,231.34,...,5,5.0,2,NaN,Rejected,0.0,NaN,0,0,50
4,36034981,32592943,0.781269,3120,3300.0,2,9900,63,3,639.45,...,4,5.0,1,0.0,Approved,NaN,0.0,0,0,174


Move 'working_months' to the first column

In [21]:
# Create a list of columns without 'working_months'
df_sorted_columns = df.columns.tolist()
df_sorted_columns.remove('working_months')
# Add 'working_months' to the first position
df_sorted_columns.insert(0, 'working_months')
# Reindex columns
df = df[df_sorted_columns]

In [22]:
# Update the list of date and numeric columns
date_columns = []
numeric_columns = numeric_columns + ['working_months']

In [23]:
df.head()

,working_months,ID,customer_id,_r_,Var9,Var10,Var1,Var4,Var5,Var6,...,Var3,Var11,Var12,Var14,target,Application_status,Var18,Var19,Var27,Var28
0,62,36034977,32653719,0.485674,4200,NaN,1,4800,15,1,...,1.0,2,NaN,0,0.0,Approved,NaN,NaN,0,0
1,160,36034978,32832365,0.267045,5880,3700.0,2,6800,18,6,...,1.0,5,6.0,3,0.0,Approved,1.0,NaN,0,0
2,134,36034979,32544742,0.793579,4200,NaN,1,4600,18,1,...,1.0,5,NaN,0,0.0,Approved,NaN,NaN,0,0
3,50,36034980,32395830,0.533040,4200,5400.0,2,7900,48,1,...,2.0,5,5.0,2,NaN,Rejected,0.0,NaN,0,0
4,174,36034981,32592943,0.781269,3120,3300.0,2,9900,63,3,...,1.0,4,5.0,1,0.0,Approved,NaN,0.0,0,0


### Handle binary variables

Convert the 'Application_status' column to binary variables to facilitate analysis. Applications marked as 'Approved' are changed to 1, and those marked as 'Rejected' are changed to 0. This conversion simplifies the analysis process, allowing for straightforward interpretation of application outcomes

In [24]:
# Check unique values for binary columns
for column in binary_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'target': [ 0. nan  1.]
Unique values for column 'Application_status': ['Approved' 'Rejected']
Unique values for column 'Var18': [nan  1.  0.]
Unique values for column 'Var19': [nan  0.  1.]
Unique values for column 'Var27': [0 1]
Unique values for column 'Var28': [0 1]


As we can observe column "Application_status" needs to be modified

In [25]:
# Change value "Approved" to 1 and "Rejected" to 0 in column 'Application_status'
df['Application_status'] = df['Application_status'].replace({'Approved': 1, 'Rejected': 0})

In [26]:
# Check and count unique values for 'Application_status' to ensure that transformation was successful
df['Application_status'].value_counts(dropna=False)

1    3647
0    1353
Name: Application_status, dtype: int64

Check the rest of binary columns

In [27]:
df[binary_columns].isna().sum()

target                1353
Application_status       0
Var18                 3721
Var19                 2905
Var27                    0
Var28                    0
dtype: int64

In [28]:
df[binary_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   target              3647 non-null   float64
 1   Application_status  5000 non-null   int64  
 2   Var18               1279 non-null   float64
 3   Var19               2095 non-null   float64
 4   Var27               5000 non-null   int64  
 5   Var28               5000 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 234.5 KB


We can observe that we have some nulls in binary variables, but we will handle them later.

### Handle categorical variables

We have 5 categorical columns:

* Var2 - Loan purpose
* Var3 - Distribution channel
* Var11 - profession of main applicant
* Var12 - profession of second applicant
* Var14 - marital status of main applicant

All of them are treated as simple categorical variables(non-ordered)

In [29]:
# Check unique values for binary columns
for column in categorical_columns:
    print(f"Unique values for column '{column}': {df[column].unique()}")

Unique values for column 'Var2': [ 3.  2.  1. nan]
Unique values for column 'Var3': [ 1.  2. nan  3.]
Unique values for column 'Var11': [2 5 4 6 3 7 1]
Unique values for column 'Var12': [nan  6.  5.  3.  4.  2.  1.  7.]
Unique values for column 'Var14': [0 3 2 1 4]


According to the variables_description.xlsx file the variable Var2 should have the following values:

* 1 Direct
* 2 Broker
* 3 Online

In the dataset this variable has 5 unique values printed above. First of all we should transform 'Direct' to 1 and 'Online' to 3

In [30]:
# Check the number of observations in Var3 for each category
df['Var3'].value_counts(dropna=False)

1.0    2454
2.0    1645
3.0     762
NaN     139
Name: Var3, dtype: int64

In [31]:
# Change the value "Direct" to 1 and "Online" to 3 in column "Var3"
df['Var3'] = df['Var3'].replace({'Direct': 1, 'Online': 3})

# Change the data type of column "Var3" to float (we still have NaN values)
df['Var3'] = df['Var3'].astype(float)

Check the unique values after transformation

In [32]:
# Check again the number of observations in Var3 for each category
df['Var3'].value_counts(dropna=False)

1.0    2454
2.0    1645
3.0     762
NaN     139
Name: Var3, dtype: int64

It looks that the transformation was successfull

In [33]:
# Count NA values for categorical columns
df[categorical_columns].isna().sum()

Var2      139
Var3      139
Var11       0
Var12    3719
Var14       0
dtype: int64

All categorical variables are ready for the further processing (filling NA values and one hot encoding).

### Handle NaN in target variable

As we can observe below we have 13282 NaN values in our target variable.

We found out that all of them are for "Application status" == rejected (1).

We analyze the approach to fill target variable of all rejected application to 1 (default). However this way would bring in a high conlusion that all of rejected people would default in the future and it would impact negatively on our model. Additionally we assume that we want to model the approved applications only.

After a long discussion we decided to delete all of rows with Application_status == 0.

In [34]:
# Check the number of NAa for target variable
df['target'].isna().sum()

1353

In [35]:
df.loc[df["Application_status"] == 0, "target"].isna().sum()

1353

In [36]:
# Drop rows where 'Application_status' == 0
df = df[df['Application_status'] != 0]

Now as we know that there are only rows with application_status == 0 we can delete this column from df

In [37]:
# Drop column 'Application_status'
df.drop(['Application_status'], axis=1, inplace=True)

In [38]:
# Delete "application_stauts" from binary_columns
binary_columns.remove('Application_status')

### Handle numeric data

In [39]:
df[numeric_columns].isna().sum()

ID                   0
customer_id          0
_r_                  0
Var9                 0
Var10             2757
Var1                 0
Var4                 0
Var5                 0
Var6                 0
Var7                 0
Var8              2030
Var15                0
Var16                0
Var17                0
Var20                0
Var21                0
Var22                0
Var23                0
Var24                0
Var25              722
Var26             1415
Var29                0
Var30                0
working_months       0
dtype: int64

#### Dependence of numeric variables with categorical variables

Var10 and Var8 are strictly related to Var1 and Var2 respectively, so we wanted to check the NAs in dependent variables.
However Var1 related to Var10 does not have any missings we moved to Var8 and Var2.

We wanted to check if there are any NA observations of Var2 (Loan purpose) having Var8 (Value of goods(car)) set to some value. If yes we assume that the value is entered correctly and we are missing Var2 for some unknown reason.

We decided to fill these rows with value 1 (meaning Car Loan).

As we can observe below we have 1018 NAs before filling the data.

In [40]:
# Count values inside Var8 group by value
df['Var2'].value_counts(dropna=False)

1.0    1575
3.0    1011
2.0     960
NaN     101
Name: Var2, dtype: int64

In [41]:
# Put value 1 to the column Var2 if Var8 is not NaN
df.loc[~df['Var8'].isna(), 'Var2'] = 1

After transformation the number of NaN in Var8 dropped by almost half

In [42]:
# count values inside Var8 group by value
df['Var2'].value_counts(dropna=False)

1.0    1617
3.0    1011
2.0     960
NaN      59
Name: Var2, dtype: int64

#### Analyze Var9 and Var10

We have 28043 null values in Var10 (income of second applicant). We would like to check how many of these NAs is because of the fact that there is just one applicant.

As we can see below all of NAs are inside this group. We decided to sum these columns into one and name it "income".

In [43]:
# Count the number of nulls where Var1 = 1
df.loc[df["Var1"] == 1, "Var10"].isna().sum()

2757

In [44]:
# Create a new colum income
df["income"] = df["Var9"] + df["Var10"].fillna(0).astype(int) # Var9 was as integer, Var10 was as float, but we do not have NAs

In [45]:
# Drop columns Var9 and Var10
df.drop(["Var9", "Var10"], axis=1, inplace=True)

In [46]:
# update the list of numeric columns
numeric_columns.remove("Var9")
numeric_columns.remove("Var10")
numeric_columns.append("income")

Check again the number of null values

In [47]:
df[numeric_columns].isna().sum()

ID                   0
customer_id          0
_r_                  0
Var1                 0
Var4                 0
Var5                 0
Var6                 0
Var7                 0
Var8              2030
Var15                0
Var16                0
Var17                0
Var20                0
Var21                0
Var22                0
Var23                0
Var24                0
Var25              722
Var26             1415
Var29                0
Var30                0
working_months       0
income               0
dtype: int64

We still have a few variables with nulls:

* Var8 - Value of the goods (car) - We decided to delete this variables from the dataset. The number of NAs is relatively huge (almost 56%), the variables refers only to the loans for car purposes and filling this with 0, mode, median or using KNN would not hane any sense.
* Var17 - Spendings estimation - the number of NAs is low, there is a sense in filling this, we decided to use KNN to fill these variables.
* Var25 - Amount on current account 
* Var26 - Amount on savings account

When it comes to Var25 and Var26 we decidec to fill NAa with 0. NAs means probably that the client does not have account in our bank and we cannot reach that value or does not have any money on either current or saving account. In both cases there is an element of uncertainty which we decided to minimize.

Because of this our model should be more conservative and penalize clients about whom we are not sure.

In [48]:
# Check the percentage number of NAs in Var8 related to whole df
round(df['Var8'].isna().sum() / df.shape[0] * 100, 2)

55.66

In [49]:
# Drop Var8 column
df.drop('Var8', axis=1, inplace=True)

In [50]:
# Drop Var8 from the list of numeric columns
numeric_columns.remove('Var8')

In [51]:
# Put a value 0 to the NaN in Var25 and Var26 columns
df['Var25'] = df['Var25'].fillna(0)
df['Var26'] = df['Var26'].fillna(0)

In [52]:
df[numeric_columns].isna().sum()

ID                0
customer_id       0
_r_               0
Var1              0
Var4              0
Var5              0
Var6              0
Var7              0
Var15             0
Var16             0
Var17             0
Var20             0
Var21             0
Var22             0
Var23             0
Var24             0
Var25             0
Var26             0
Var29             0
Var30             0
working_months    0
income            0
dtype: int64

### Handle remaining NULL values

In [53]:
df.isnull().sum()

working_months       0
ID                   0
customer_id          0
_r_                  0
Var1                 0
Var4                 0
Var5                 0
Var6                 0
Var7                 0
Var15                0
Var16                0
Var17                0
Var20                0
Var21                0
Var22                0
Var23                0
Var24                0
Var25                0
Var26                0
Var29                0
Var30                0
Var2                59
Var3               101
Var11                0
Var12             2757
Var14                0
target               0
Var18             2656
Var19             2030
Var27                0
Var28                0
income               0
dtype: int64

* Var17 - spedning estimation - as we wrote before, we will fill with KNN
* Var2 - loan purpose [categorical] - drop NA rows
* Var3 - Distribution channel [categorical] - drop NA rows
* Var12 - profession of second applicant [categorical] - add new category
* Var18 - Property ownership for property renovation
* Var19 - Clasification of the vehicle (Car, Motorbike)

When it comes to the Var18 and Var19 we analyze combining them with Var2 (loan purpose) and have 1 categorical variable with values:

1. Car loan motorbike
2. Car loan car
3. House renovation your property
4. House renovation not your property
5. Short cash

We finally decided not to use this approach, however we provide you with the short analysis why. 

#### Var12

Check if all NAs in Var12 (profession of second applicant)are if Var1 == 1, so if there is just one applicant.

The number of NAs in this category is the same as the number of NAs when Var1 == 1, so we fill it with a new category.

In [54]:
df.loc[df["Var1"] == 1, "Var12"].isna().sum()

2757

Fill NA in Var12 (profession of second applicant) with new category: 8 - just one applicant

In [55]:
df["Var12"] = df["Var12"].fillna(8)

#### Var18 and Var19 - check

Check if we can combine Var2 with Var18 and Var19 and having one categorical variable instead of one vategorical and 2 binary with huge amount of nulls. The new variable would consists of 5 values:

1. Car loan motorbike
2. Car loan car
3. House renovation your property
4. House renovation not your property
5. Short cash

First of all we need to check the values - if combining them would reduce the nulls to 0 and nulls are related only to the variable specificity or we really have some missing data.

Var2 == 1 is related to Var 19 and Var2 == 2 is related to Var 18.

In [56]:
df.shape[0]

3647

In [57]:
# Check if the number of "conditional" values is the same as the number of rows in df

# Count rows where Var2 == 1 and Var19 is not NaN
var2_19 = df[(df['Var2'] == 1) & (~df['Var19'].isna())].shape[0]

# Count rows where Var2 == 2 and Var 18 is not NaN
var2_18 = df[(df['Var2'] == 2) & (~df['Var18'].isna())].shape[0]

# Count rows where Var2 == 3
var2_3 = df.loc[df["Var2"] == 3].shape[0]

# Calculate of sum
sum = var2_19 + var2_18 + var2_3

print(var2_18)
print(var2_19)
print(var2_3)
print(sum)

960
1617
1011
3588


The number of rows in df is 36718 while the "conditional" sum is 36136, meaning that there would be a new category or we would need to add original values from Var2 and the number of categories would increase.

Lets try to create this categorical column and see the distribution.

In [58]:
# Add new column loan_desc
df['loan_desc'] = 0

# Check conditions
df.loc[(df['Var2'] == 1) & (df['Var19'] == 1), 'loan_desc'] = 1
df.loc[(df['Var2'] == 1) & (df['Var19'] == 0), 'loan_desc'] = 2
df.loc[(df['Var2'] == 2) & (df['Var18'] == 1), 'loan_desc'] = 3
df.loc[(df['Var2'] == 2) & (df['Var18'] == 0), 'loan_desc'] = 4
df.loc[df['Var2'] == 3, 'loan_desc'] = 5

In [59]:
df['loan_desc'].value_counts(dropna=False)

1    1183
5    1011
3     801
2     434
4     159
0      59
Name: loan_desc, dtype: int64

As we can observe above introducing new variables diveded the df into smaller, not very numerous groups. Vast majority of data are concentrated around 1, 5 and 3 group so probably (there was no precise description of variable Var18 and Var19) for:

 * Car loan for car
 * House renovation for client's property
 * Short cash

These new two groups (2, 4) and additional 0 constitute a small percentage of all df.

Summarizing we decide to go with variable Var2 and delete Var18 and Var19.

In [60]:
df.shape

(3647, 33)

#### Drop Var18 i Var19

In [61]:
# Drop Var18 and Var19 columns
df.drop('Var18', axis=1, inplace=True)
df.drop('Var19', axis=1, inplace=True)

In [62]:
# Update the list of binary columns
binary_columns.remove('Var18')
binary_columns.remove('Var19')

#### Var2 and Var3 - drop NAs

In [63]:
# Drop NA rows fof Var2
df = df[~df['Var2'].isna()]
df = df[~df['Var3'].isna()]

In [64]:
# sum number of observations for each category in Var3
df['Var3'].value_counts(dropna=False)

1.0    1817
2.0    1240
3.0     489
Name: Var3, dtype: int64

In [65]:
df.isna().sum()

working_months    0
ID                0
customer_id       0
_r_               0
Var1              0
Var4              0
Var5              0
Var6              0
Var7              0
Var15             0
Var16             0
Var17             0
Var20             0
Var21             0
Var22             0
Var23             0
Var24             0
Var25             0
Var26             0
Var29             0
Var30             0
Var2              0
Var3              0
Var11             0
Var12             0
Var14             0
target            0
Var27             0
Var28             0
income            0
loan_desc         0
dtype: int64

Summarizing we still have 31 NAs in Var17 which will be filled using KNN

### One hot encoding to categorical columns

Apply one-hot encoding to the specified categorical nominal variables in the 'sorted_data' DataFrame.
The variables identified for this encoding are: Var2, Var3, Var11, Var12, and Var14.
One-hot encoding converts these categorical variables into a form that could be provided to ML algorithms
to do a better job in prediction. It creates binary columns for each category/value in the original columns
and assigns a 1 or 0 (presence or absence) in those newly created binary columns.

The 'pd.get_dummies()' function is used here to perform one-hot encoding on the specified columns.
It automatically generates new columns for each unique value in the specified categorical columns,
effectively transforming them into a format that's suitable for modeling.

In [66]:
df.head()

,working_months,ID,customer_id,_r_,Var1,Var4,Var5,Var6,Var7,Var15,...,Var2,Var3,Var11,Var12,Var14,target,Var27,Var28,income,loan_desc
0,62,36034977,32653719,0.485674,1,4800,15,1,463.31,1,...,3.0,1.0,2,8.0,0,0.0,0,0,4200,5
1,160,36034978,32832365,0.267045,2,6800,18,6,3170.40,0,...,2.0,1.0,5,6.0,3,0.0,0,0,9580,3
2,134,36034979,32544742,0.793579,1,4600,18,1,355.36,0,...,3.0,1.0,5,8.0,0,0.0,0,0,4200,5
4,174,36034981,32592943,0.781269,2,9900,63,3,639.45,0,...,1.0,1.0,4,5.0,1,0.0,0,0,6420,2
5,196,36034982,32601182,0.861555,2,13700,33,1,562.17,0,...,1.0,1.0,5,5.0,0,0.0,0,0,7240,2


In [67]:
categorical_columns

['Var2', 'Var3', 'Var11', 'Var12', 'Var14']

In [68]:
df = pd.get_dummies(df, columns=['Var2', 'Var3', 'Var11', 'Var12', 'Var14'], drop_first=True)

# Display the first few rows of the DataFrame to check the results of the encoding.
# This step allows us to verify that the one-hot encoding process has successfully transformed the specified
# categorical variables into a set of binary variables, each representing a unique category within the original variables.
df.head()

,working_months,ID,customer_id,_r_,Var1,Var4,Var5,Var6,Var7,Var15,...,Var12_3.0,Var12_4.0,Var12_5.0,Var12_6.0,Var12_7.0,Var12_8.0,Var14_1,Var14_2,Var14_3,Var14_4
0,62,36034977,32653719,0.485674,1,4800,15,1,463.31,1,...,0,0,0,0,0,1,0,0,0,0
1,160,36034978,32832365,0.267045,2,6800,18,6,3170.40,0,...,0,0,0,1,0,0,0,0,1,0
2,134,36034979,32544742,0.793579,1,4600,18,1,355.36,0,...,0,0,0,0,0,1,0,0,0,0
4,174,36034981,32592943,0.781269,2,9900,63,3,639.45,0,...,0,0,1,0,0,0,1,0,0,0
5,196,36034982,32601182,0.861555,2,13700,33,1,562.17,0,...,0,0,1,0,0,0,0,0,0,0


In [69]:
df.isna().sum()

working_months    0
ID                0
customer_id       0
_r_               0
Var1              0
Var4              0
Var5              0
Var6              0
Var7              0
Var15             0
Var16             0
Var17             0
Var20             0
Var21             0
Var22             0
Var23             0
Var24             0
Var25             0
Var26             0
Var29             0
Var30             0
target            0
Var27             0
Var28             0
income            0
loan_desc         0
Var2_2.0          0
Var2_3.0          0
Var3_2.0          0
Var3_3.0          0
Var11_2           0
Var11_3           0
Var11_4           0
Var11_5           0
Var11_6           0
Var11_7           0
Var12_2.0         0
Var12_3.0         0
Var12_4.0         0
Var12_5.0         0
Var12_6.0         0
Var12_7.0         0
Var12_8.0         0
Var14_1           0
Var14_2           0
Var14_3           0
Var14_4           0
dtype: int64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3546 entries, 0 to 4999
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  3546 non-null   int32  
 1   ID              3546 non-null   int64  
 2   customer_id     3546 non-null   int64  
 3   _r_             3546 non-null   float64
 4   Var1            3546 non-null   int64  
 5   Var4            3546 non-null   int64  
 6   Var5            3546 non-null   int64  
 7   Var6            3546 non-null   int64  
 8   Var7            3546 non-null   float64
 9   Var15           3546 non-null   int64  
 10  Var16           3546 non-null   int64  
 11  Var17           3546 non-null   float64
 12  Var20           3546 non-null   int64  
 13  Var21           3546 non-null   int64  
 14  Var22           3546 non-null   int64  
 15  Var23           3546 non-null   int64  
 16  Var24           3546 non-null   int64  
 17  Var25           3546 non-null   f

In [71]:
# change working_months to int32
df['working_months'] = df['working_months'].astype('int32')

In [72]:
# Set the seed for reproducibility
np.random.seed(123456)

# Save the original data types of the DataFrame. This is crucial for restoring the DataFrame's data types
# after imputation since KNNImputer will convert the imputed data to float64 by default.
original_dtypes = df.dtypes

# Initialize the KNNImputer. The default setting uses 5 nearest neighbors to impute missing values,
# which is a balance between accuracy and computational efficiency.
knn_imputer = KNNImputer()

# Replace 0s with NaNs in the 'working_months' column if necessary. This step is preparatory for imputation,
# ensuring that 0s, which may signify missing or unrecorded data, are correctly treated as missing values.
df['working_months'] = df['working_months'].replace(0, np.nan)

# Select only numerical columns from the DataFrame for imputation. KNNImputer works with numerical data,
# so identifying these columns is a prerequisite for the imputation process.
numerical_cols = df.select_dtypes(include=['int64', 'float64', 'int32']).columns

# Perform the imputation using KNN. This process fills in missing values based on the mean of the
# nearest neighbors found in the multidimensional space, providing a reasonable estimate for missing data.
imputed_data = knn_imputer.fit_transform(df[numerical_cols])

# Convert the imputed array back into a DataFrame, preserving the original index. This step ensures that
# the imputed data retains the structure and index of the original DataFrame for consistency and ease of analysis.
imputed_df = pd.DataFrame(imputed_data, columns=numerical_cols, index=df.index)

# Restore the original data types. After imputation, it's important to convert the data back to its
# original types for accurate analysis and consistency with the rest of the dataset.
for col in numerical_cols:
    if original_dtypes[col].kind in 'iu':  # Check if the data type is integer
        imputed_df[col] = imputed_df[col].round().astype(original_dtypes[col])
    else:  # For floating-point types, retain as is
        imputed_df[col] = imputed_df[col].astype(original_dtypes[col])

# Update the original 'sorted_data_encoded' DataFrame with the imputed data. This final step integrates
# the imputed numerical data back into the main DataFrame, completing the imputation process.
df.update(imputed_df)

# Display the DataFrame information to check the results of the imputation and data type restoration.
# This overview allows for a quick verification that the dataset is now complete with imputed values
# and that all columns are of their appropriate data types.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3546 entries, 0 to 4999
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  3546 non-null   float64
 1   ID              3546 non-null   int64  
 2   customer_id     3546 non-null   int64  
 3   _r_             3546 non-null   float64
 4   Var1            3546 non-null   int64  
 5   Var4            3546 non-null   int64  
 6   Var5            3546 non-null   int64  
 7   Var6            3546 non-null   int64  
 8   Var7            3546 non-null   float64
 9   Var15           3546 non-null   int64  
 10  Var16           3546 non-null   int64  
 11  Var17           3546 non-null   float64
 12  Var20           3546 non-null   int64  
 13  Var21           3546 non-null   int64  
 14  Var22           3546 non-null   int64  
 15  Var23           3546 non-null   int64  
 16  Var24           3546 non-null   int64  
 17  Var25           3546 non-null   f

### Final check of df

In [73]:
df.head()

,working_months,ID,customer_id,_r_,Var1,Var4,Var5,Var6,Var7,Var15,...,Var12_3.0,Var12_4.0,Var12_5.0,Var12_6.0,Var12_7.0,Var12_8.0,Var14_1,Var14_2,Var14_3,Var14_4
0,62.0,36034977,32653719,0.485674,1,4800,15,1,463.31,1,...,0,0,0,0,0,1,0,0,0,0
1,160.0,36034978,32832365,0.267045,2,6800,18,6,3170.40,0,...,0,0,0,1,0,0,0,0,1,0
2,134.0,36034979,32544742,0.793579,1,4600,18,1,355.36,0,...,0,0,0,0,0,1,0,0,0,0
4,174.0,36034981,32592943,0.781269,2,9900,63,3,639.45,0,...,0,0,1,0,0,0,1,0,0,0
5,196.0,36034982,32601182,0.861555,2,13700,33,1,562.17,0,...,0,0,1,0,0,0,0,0,0,0


In [74]:
df.isna().sum()

working_months    0
ID                0
customer_id       0
_r_               0
Var1              0
Var4              0
Var5              0
Var6              0
Var7              0
Var15             0
Var16             0
Var17             0
Var20             0
Var21             0
Var22             0
Var23             0
Var24             0
Var25             0
Var26             0
Var29             0
Var30             0
target            0
Var27             0
Var28             0
income            0
loan_desc         0
Var2_2.0          0
Var2_3.0          0
Var3_2.0          0
Var3_3.0          0
Var11_2           0
Var11_3           0
Var11_4           0
Var11_5           0
Var11_6           0
Var11_7           0
Var12_2.0         0
Var12_3.0         0
Var12_4.0         0
Var12_5.0         0
Var12_6.0         0
Var12_7.0         0
Var12_8.0         0
Var14_1           0
Var14_2           0
Var14_3           0
Var14_4           0
dtype: int64

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3546 entries, 0 to 4999
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   working_months  3546 non-null   float64
 1   ID              3546 non-null   int64  
 2   customer_id     3546 non-null   int64  
 3   _r_             3546 non-null   float64
 4   Var1            3546 non-null   int64  
 5   Var4            3546 non-null   int64  
 6   Var5            3546 non-null   int64  
 7   Var6            3546 non-null   int64  
 8   Var7            3546 non-null   float64
 9   Var15           3546 non-null   int64  
 10  Var16           3546 non-null   int64  
 11  Var17           3546 non-null   float64
 12  Var20           3546 non-null   int64  
 13  Var21           3546 non-null   int64  
 14  Var22           3546 non-null   int64  
 15  Var23           3546 non-null   int64  
 16  Var24           3546 non-null   int64  
 17  Var25           3546 non-null   f

#### Write df as csv

In [76]:
df = df.drop('loan_desc', axis=1)

In [77]:
# Załóżmy, że mamy DataFrame df z kolumnami, w tym 'income' i 'target'

# Pobierz listę kolumn, wyjąwszy 'income', którą będziemy przesuwać
cols = [col for col in df.columns if col != 'income']

# Znajdź indeks kolumny 'target'
target_index = cols.index('target')

# Wstaw 'income' bezpośrednio przed 'target'
cols.insert(target_index, 'income')

# Zastosuj nową kolejność kolumn
df = df[cols]

In [78]:
df.to_csv('../data/output/testing_sample_cleaned.csv', index = False)  # Save the cleaned data to a new CSV file